<a href="https://colab.research.google.com/github/everestso/Fall24Spring25/blob/main/UDL_pyTorch_DeepNetworks_with_DiamondsInColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning w/ Diamonds

https://www.kaggle.com/datasets/shivam2503/diamonds

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
# Define a standardization scaler to transform values
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

import time

In [ ]:
!gdown 1-OzEC9axnc2fTMMcHCXnBkQzypluJLLG

file_name = 'diamonds.csv'

df = pd.read_csv(file_name)

df.head()
print (df.shape)

Downloading...
From: https://drive.google.com/uc?id=1-OzEC9axnc2fTMMcHCXnBkQzypluJLLG
To: /content/diamonds.csv
100% 2.57M/2.57M [00:00<00:00, 18.5MB/s]
(53940, 10)


# 3 Feature Model

In [ ]:
#diamond_sample = df.sample(30, random_state=12)
diamond_sample = df.copy()
print(f"{diamond_sample.shape=}")

# Create a dataframe X containing all the features except carat, cut, and color
drop_columns = ['depth', 'table', 'x', 'y', 'z', 'price']
X = diamond_sample.drop(drop_columns, axis=1)
print(f"{X.columns=}, {X.shape=}")
print(f"{X[:3]}")


# Create a dataframe y containing the feature price
y = diamond_sample[['price']]
print(f"{y.columns}")
print(f"{y.iloc[:3]=}")

diamond_sample.shape=(53940, 10)
X.columns=Index(['carat', 'cut', 'color', 'clarity'], dtype='object'), X.shape=(53940, 4)
   carat      cut color clarity
0   0.23    Ideal     E     SI2
1   0.21  Premium     E     SI1
2   0.23     Good     E     VS1
Index(['price'], dtype='object')
y.iloc[:3]=   price
0    326
1    326
2    327


## Transforms

In [ ]:

# Split data into train and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=123)

# Define a dictionary for quality mapping
cut_quality = {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium': 4, 'Ideal': 5}

# Apply the mapping to Xtrain and Xtest
Xtrain['cut'] = Xtrain['cut'].map(cut_quality)
Xtest['cut'] = Xtest['cut'].map(cut_quality)

# Create an OrdinalEncoder for the 'color' column
enc = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E', 'D']]) # Define the order of categories

# Fit the encoder on the training data and transform both training and testing data
Xtrain['color'] = enc.fit_transform(Xtrain[['color']])
Xtest['color'] = enc.transform(Xtest[['color']])

# Create an OrdinalEncoder for the 'clarity' column
clarity_enc = OrdinalEncoder(categories=[['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']])
Xtrain['clarity'] = clarity_enc.fit_transform(Xtrain[['clarity']])
Xtest['clarity'] = clarity_enc.transform(Xtest[['clarity']])

Transform = StandardScaler()

# Apply scaler
Xtrain_scaled = Transform.fit_transform(Xtrain)
Xtest_scaled = Transform.transform(Xtest)

print  (Xtrain_scaled)

[[ 0.55116139  0.97923439 -0.82538596 -0.02809222]
 [-0.84330851  0.97923439  0.34923047  1.18761651]
 [-0.82218018  0.08287491  0.93653868 -0.63594658]
 ...
 [-1.05459183  0.97923439  0.93653868 -0.02809222]
 [ 0.4243914   0.97923439 -0.23807775  0.57976215]
 [-0.18833022 -0.81348456 -1.41269417  1.79547088]]


In [ ]:
# prompt: Print the transform formula details

print(f"Transform.mean_ = {Transform.mean_}")
print(f"Transform.scale_ = {Transform.scale_}")

print (f"{Xtrain.columns[0]=}, {Transform.mean_[0]=}, {Transform.scale_[0]=}, {np.std(Xtrain[Xtrain.columns[0]])=}")
print (f"{Xtrain.iloc[0,0]=}, {(Xtrain.iloc[0,0]-Transform.mean_[0])/Transform.scale_[0]=}, {Xtrain_scaled[0][0]=}")
print (f"{np.mean(Xtrain, axis=0)=}")
print (f"{np.std(Xtrain, axis=0)=}")

Transform.mean_ = [0.79913634 3.90754277 3.40537105 3.04621537]
Transform.scale_ = [0.47329813 1.11562384 1.70268349 1.64513091]
Xtrain.columns[0]='carat', Transform.mean_[0]=0.7991363419672652, Transform.scale_[0]=0.47329813202512444, np.std(Xtrain[Xtrain.columns[0]])=0.47329813202512444
Xtrain.iloc[0,0]=1.06, (Xtrain.iloc[0,0]-Transform.mean_[0])/Transform.scale_[0]=0.5511613935946977, Xtrain_scaled[0][0]=0.5511613935946977
np.mean(Xtrain, axis=0)=carat      0.799136
cut        3.907543
color      3.405371
clarity    3.046215
dtype: float64
np.std(Xtrain, axis=0)=carat      0.473298
cut        1.115624
color      1.702683
clarity    1.645131
dtype: float64


# Now pyTorch

https://aibyhand.substack.com/p/6-can-you-code-a-multi-layer-perceptron

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR

import time

# Define input size, hidden layer size, output size
D_i, D_k, D_o = 4, 40, 1

batch_size = 250
total_epochs = 50
lr=[0.15, 0.1, 0.05, 0.01]
lr_i=0

# Create model with two hidden layers
model = nn.Sequential(
    nn.Linear(D_i, D_k),
    nn.ReLU(),
    nn.Linear(D_k, D_k),
    nn.ReLU(),
    nn.Linear(D_k, D_o)
)

# He initialization of weights
def weights_init(layer):
    if isinstance(layer, nn.Linear):
        nn.init.kaiming_uniform_(layer.weight)
        layer.bias.data.fill_(0.0)

model.apply(weights_init)

# Choose least squares loss function
criterion = nn.MSELoss()


optimizers = [torch.optim.SGD(model.parameters(), lr=lr[lr_i], momentum=0.9),
              torch.optim.Adam(model.parameters())]
optimizers_i = 0

# Construct SGD optimizer and initialize learning rate and momentum
optimizer = optimizers[optimizers_i]

# Decreases learning rate by half every 10 epochs
#scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

# Lets Use Diamond Data
x = torch.tensor(Xtrain_scaled, dtype=torch.float32)
y = torch.tensor(ytrain.values, dtype=torch.float32)

data_loader = DataLoader(TensorDataset(x, y), batch_size=batch_size, shuffle=True)

# Loop over the dataset 100 times
for epoch in range(total_epochs):
    epoch_loss = 0.0
    # Loop over batches
    for i, data in enumerate(data_loader):

        # Retrieve inputs and labels for this batch
        x_batch, y_batch = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        pred = model(x_batch)
        loss = criterion(pred, y_batch)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update statistics
        epoch_loss += loss.item()

    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1:5d}, Loss {epoch_loss:.3f}")

    # Tell scheduler to update learning rate
    scheduler.step()

Epoch    10, Loss 2443852209.000
Epoch    20, Loss 2439941159.000
Epoch    30, Loss 2422697154.000
Epoch    40, Loss 2438498607.000
Epoch    50, Loss 2467207017.000


In [ ]:
from sklearn.metrics import r2_score

# Assuming 'model' is your trained Keras model, 'Xtest' is your test data, and 'ytest' are the true target values
print (f"{Xtrain.shape=}")

# Convert Xtrain to PyTorch tensor
Xtrain_tensor = torch.tensor(Xtrain_scaled, dtype=torch.float32)
y_predicted = model(Xtrain_tensor)

# Convert predictions back to numpy array for r2_score
y_predicted = y_predicted.detach().numpy()
r2 = r2_score(ytrain, y_predicted)
print("R-squared Train:", r2)

# Convert Xtest to PyTorch tensor
Xtest_tensor = torch.tensor(Xtest_scaled, dtype=torch.float32)
y_predicted = model(Xtest_tensor)

# Convert predictions back to numpy array for r2_score
y_predicted = y_predicted.detach().numpy()
r2 = r2_score(ytest, y_predicted)
print("R-squared Test:", r2)

Xtrain.shape=(37758, 4)
R-squared Train: 0.9575517177581787
R-squared Test: 0.9559349417686462
